In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm.notebook import tqdm
import xml.etree.ElementTree as xet
from glob import glob

In [ ]:
!unzip /content/car_license.zip

In [4]:
for dirname, _, filenames in os.walk('/content/car_license'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [6]:
def load_images(directory):
    images = []

    for filepath in tqdm(os.listdir(directory)):
        img_path = os.path.join(directory,filepath)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224, 224))
        img = cv2.medianBlur(img, 5)
        images.append(img)
    
    images = np.array(images).reshape(len(images), 224, 224, 3)
    return images

In [ ]:
os.listdir('/content/car_license/annotations')

In [5]:
!mkdir /content/car_license/labels

In [ ]:
path = glob("../content/car_license/annotations/*.xml")

for img_path, label_path in zip(os.listdir('/content/car_license/images'), os.listdir('/content/car_license/annotations')):
    txt_path = '/content/car_license/labels/' + img_path.split('/')[-1][:-3] + 'txt'
    print(txt_path)
    with open(txt_path, 'w') as f:
      img = cv2.imread('/content/car_license/images/' + img_path)
      H, W, _ = img.shape
      data = xet.parse('/content/car_license/annotations/' + label_path)
      root = data.getroot()
      obj = root.find('object')
      labels_info = obj.find('bndbox')
      xmin = int(labels_info.find('xmin').text)
      xmax = int(labels_info.find('xmax').text)
      ymin = int(labels_info.find('ymin').text)
      ymax = int(labels_info.find('ymax').text)

      w = (xmax - xmin) / W
      h = (ymax - ymin) / H
      xc = (xmin + (xmax - xmin) / 2) / W
      yc = (ymin + (ymax - ymin) / 2) / H

      f.write('0 ' + str(xc)+' '+str(yc)+' '+str(w)+' '+str(h)+'\n')

In [ ]:
!mkdir /content/dataset
!mkdir /content/dataset/labels
!mkdir /content/dataset/images
!mkdir /content/dataset/labels/val
!mkdir /content/dataset/labels/train
!mkdir /content/dataset/images/val
!mkdir /content/dataset/images/train

In [10]:
data_root = '/content/dataset'

In [11]:
import shutil

In [ ]:
for annotate_mode in ['images', 'labels']:
    source_dir = '/content/car_license/'
    for i, file_name in enumerate(tqdm(os.listdir(os.path.join(source_dir, 'images')))):
      print(os.path.join(source_dir, annotate_mode, file_name))
      if annotate_mode == 'labels':
        file_name = file_name[:-3] + 'txt'
      if i % 3:
        shutil.copy(os.path.join(source_dir, annotate_mode, file_name), os.path.join(data_root, annotate_mode, 'train', file_name))
      else:
        shutil.copy(os.path.join(source_dir, annotate_mode, file_name), os.path.join(data_root, annotate_mode, 'val', file_name))

In [13]:
print('Test:', len(os.listdir('/content/dataset/images/val')))
print('Train:', len(os.listdir('/content/dataset/images/train')))

Test: 145
Train: 288


In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

In [22]:
!python yolov5/train.py --img 896 --batch 8 --epoch 300 --data /content/sample_data/car_license.yaml --weights /content/sample_data/yolov5s.pt --name result --workers 2

train: weights=/content/sample_data/yolov5s.pt, cfg=, data=/content/sample_data/car_license.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=8, imgsz=896, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=yolov5/runs/train, name=result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-187-g5ef69ef Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2,